In [2]:
import xml.etree.ElementTree as ET
import pandas as pd

In [3]:
# XML tree 불러오기
tree = ET.parse("2024 drugbank full database.xml")

In [ ]:
# 불러온 파일의 root 태그 찾기
root = tree.getroot()

print(root.tag)
print(root.attrib)

In [ ]:
# root의 직접적인 자식 요소 출력
for child in root:
    print(child.tag, child.attrib)

In [ ]:
# 첫번째 drug element 아래의 자식 요소 확인
first_drug = root.find("{http://www.drugbank.ca}drug")
print(first_drug)

# element는 직접적인 자식요소들을 순회할 수 있도록 구성되어있음
for child in first_drug:
    print(child.tag) # 첫 번째 <drug> 요소의 자식들의 태그 이름을 출력

In [ ]:
id = first_drug.find("{http://www.drugbank.ca}drugbank-id")

for child in root.findall("{http://www.drugbank.ca}drug"):
    id = child.find("{http://www.drugbank.ca}drugbank-id[@primary='true']")
    print(id.text)


In [49]:
#Drug table
data_id = []
data_name = []
data_description = []
data_cas_number = []
data_average_mass = []
data_monoisotopic_mass = []
data_state = []

# 루트 요소 아래에 위치한 모든 <drug> 요소 탐색 -> 모든 <drug>요소를 리스트로 변환
for child in root.findall("{http://www.drugbank.ca}drug"):
    # 각 <drug> 요소의 자식 요소 중에서 {http://www.drugbank.ca}name 태그를 가진 첫 번째 요소 탐색
    id = child.find("{http://www.drugbank.ca}drugbank-id[@primary='true']")
    name = child.find("{http://www.drugbank.ca}name")
    description = child.find("{http://www.drugbank.ca}description")
    cas_number = child.find("{http://www.drugbank.ca}cas-number")
    average_mass = child.find("{http://www.drugbank.ca}average-mass") # 예외처리 필요
    state = child.find("{http://www.drugbank.ca}state") # 예외처리 필요
    monoisotopic_mass = child.find("{http://www.drugbank.ca}monoisotopic-mass") # 예외처리 필요
    # 태그가 있는데 내용이 없는 경우 예외처리를 안해도 None값으로 들어감
    # 아예 태그가 없는 경우 즉 객체 자체가 없는 경우는 예외처리가 필요 이 때, attrib 에러 발생

    data_id.append(id.text if id is not None else None)
    data_name.append(name.text if name is not None else None)
    data_description.append(description.text if description is not None else None)
    data_cas_number.append(cas_number.text if cas_number is not None else None)
    data_average_mass.append(average_mass.text if average_mass is not None else None)
    data_monoisotopic_mass.append(monoisotopic_mass.text if monoisotopic_mass is not None else None)
    data_state.append(state.text if state is not None else None)

    # print(id.text, name.text, monoisotopic_mass.text if monoisotopic_mass is not None else monoisotopic_mass) # 자식요소의 값 확인

df_id = pd.DataFrame({"drugbank_id":data_id})
df_name = pd.DataFrame({"name":data_name})
df_description = pd.DataFrame({"description":data_description})
df_cas_number = pd.DataFrame({"cas_number":data_cas_number})
df_average_mass = pd.DataFrame({"average_mass":data_average_mass})
df_monoisotopic_mass = pd.DataFrame({"monoisotopic_mass":data_monoisotopic_mass})
df_state = pd.DataFrame({"state":data_state})

drug = pd.concat([df_id, df_name, df_description, df_cas_number, df_average_mass, df_monoisotopic_mass, df_state], axis=1)
# print(drug.head())
drug.to_csv("drug.csv", index=False)

In [ ]:
#patent table
# drug -> patents -> patent -> patent.element
# 두번 탐색 필요. 첫 번째 탐색하면서 patents가 있는지 없는지 여부 판단.
# 존재한다면 순회하면서 리스트에 값 추가.
# 존재하지않는다면 나머지 리스트에 NUll값 추가.
# 다만 테이블을 쪼개지 않고 이렇게 파싱하면 비효율적이지 않나.
# drugbank_id와 patent_number가 튜플 값을 결정하기때문에 table을 나누는게 효율적이라 판단.
# -- erd --
# patent number가 NULL값이 있으므로 PK가 될 수없다.
# 즉 drugbank_id가 PK이자 FK가 된다. 
# drugbank_id -> patent_number
# patent_number -> country, approved, expires, pediatric_extension 으로 정규화 가능

data_patent_number = []
data_p_id = []
data_country = []
data_approved = []
data_expires = []
data_pdiatric_extension = []

for child in root.findall("{http://www.drugbank.ca}drug"):
    id = child.find("{http://www.drugbank.ca}drugbank-id[@primary='true']")
    
    # print("-----------------------")

    patents = child.find("{http://www.drugbank.ca}patents")

    # if patents.text is None:
    #     data_p_id.append(id.text)
    #     # print(id.text)
    #     data_patent_number.append(None)
    #     # print("NULL")
    #     data_country.append(None)
    #     # print("NULL")
    #     data_approved.append(None)
    #     # print("NULL")
    #     data_expires.append(None)
    #     # print("NULL")
    #     data_pdiatric_extension.append(None)
    #     # print("NULL")

    if patents is not None:
        for patent in patents.findall("{http://www.drugbank.ca}patent"):
            number = patent.find("{http://www.drugbank.ca}number")
            country = patent.find("{http://www.drugbank.ca}country")
            approved = patent.find("{http://www.drugbank.ca}approved")
            expires = patent.find("{http://www.drugbank.ca}expires")
            pediatric_extension = patent.find("{http://www.drugbank.ca}pediatric-extension")

            data_p_id.append(id.text)
            data_patent_number.append(number.text)
            data_country.append(country.text)
            data_approved.append(approved.text)
            data_expires.append(expires.text)
            data_pdiatric_extension.append(pediatric_extension.text)
            # print(id.text)
            # print(number.text)
            # print(country.text)
            # print(approved.text)
            # print(expires.text)
            # print(pediatric_extension.text)

# Transpose_dataframe 씌우기
df_p_id = pd.DataFrame({"drugbank_id":data_p_id})
df_patent_number = pd.DataFrame({"patent_number":data_patent_number})
df_country = pd.DataFrame({"country":data_country})
df_approved = pd.DataFrame({"approved":data_approved})
df_expires = pd.DataFrame({"expires":data_expires})
df_pediatric_extension = pd.DataFrame({"pediatric_extension":data_pdiatric_extension})

# 컬럼 합치기
Patent = pd.concat([df_p_id, df_patent_number, df_country, df_approved, df_expires, df_pediatric_extension], axis=1)
print(Patent.head(50))

# 파일 내보내기
Patent.to_csv("patent.csv", index=False)

In [12]:
#calculated_properties
# kind가 value, source를 결정하는게 아님.
# drugbank_id -> kind, value, source

data_c_id = []
data_kind = []
data_value = []
data_source = []

for child in root.findall("{http://www.drugbank.ca}drug"):
    id = child.find("{http://www.drugbank.ca}drugbank-id[@primary='true']")

    calculated_properties = child.find("{http://www.drugbank.ca}calculated-properties")

    # if calculated_properties is None:
    #     data_c_id.append(id.text)
    #     data_kind.append(None)
    #     data_value.append(None)
    #     data_source.append(None)

    if calculated_properties is not None:
        for property in calculated_properties.findall("{http://www.drugbank.ca}property"):
            kind = property.find("{http://www.drugbank.ca}kind")
            value = property.find("{http://www.drugbank.ca}value")
            source = property.find("{http://www.drugbank.ca}source")

            data_c_id.append(id.text)
            data_kind.append(kind.text)
            data_value.append(value.text)
            data_source.append(source.text)
            
df_c_id = pd.DataFrame({"drugbank_id":data_c_id})
df_kind = pd.DataFrame({"kind":data_kind})
df_value = pd.DataFrame({"value":data_value})
df_source = pd.DataFrame({"source":data_source})

cp = pd.concat([df_c_id, df_kind, df_value, df_source], axis=1)
cp.to_csv("calculated_properties.csv", index=False)

# print(cp.head(5))

In [ ]:
#general_reference

# --- article ----------
data_a_id = []
data_ref_id_a = []
data_pubmed_id = []
data_citation = []

# ---- link ------------
data_l_id = []
data_ref_id_l = []
data_title = []
data_url = []

for child in root.findall("{http://www.drugbank.ca}drug"):
    id = child.find("{http://www.drugbank.ca}drugbank-id[@primary='true']")

    general_references = child.find("{http://www.drugbank.ca}general-references")
    # print("--------------------------")
    # print(id.text)
    # print(general_references)
    #general_reference는 모두 갖고 있음


    articles = general_references.find("{http://www.drugbank.ca}articles")
    links = general_references.find("{http://www.drugbank.ca}links")

    for article in articles.findall("{http://www.drugbank.ca}article"):

        ref_id_a = article.find("{http://www.drugbank.ca}ref-id")
        pubmed_id = article.find("{http://www.drugbank.ca}pubmed-id")
        citation = article.find("{http://www.drugbank.ca}citation")

        data_a_id.append(id.text)
        data_ref_id_a.append(ref_id_a.text)
        data_pubmed_id.append(pubmed_id.text)
        data_citation.append(citation.text)

        # print(id.text)
        # print(ref_id_a.text)
        # print(pubmed_id.text)
        # print(citation.text)

    for link in links.findall("{http://www.drugbank.ca}link"):
        
        ref_id_l = link.find("{http://www.drugbank.ca}ref-id")
        title = link.find("{http://www.drugbank.ca}title")
        url = link.find("{http://www.drugbank.ca}url")

        data_l_id.append(id.text)
        data_ref_id_l.append(ref_id_l.text)
        data_title.append(title.text)
        data_url.append(url.text)

        # print(id.text)
        # print(ref_id_l.text)
        # print(title.text)
        # print(url.text)

# -------- article ----------
df_a_id = pd.DataFrame({"drugbank_id":data_a_id})
df_ref_id_a = pd.DataFrame({"ref_id_a":data_ref_id_a})
df_pubmed_id = pd.DataFrame({"pubmed_id":data_pubmed_id})
df_citation = pd.DataFrame({"citation":data_citation})

ac = pd.concat([df_a_id, df_ref_id_a, df_pubmed_id, df_citation], axis=1)

ac.to_csv("article.csv", index=False)

# -------- link ---------------
df_l_id = pd.DataFrame({"drugbank_id":data_l_id})
df_ref_id_l = pd.DataFrame({"ref_id_l":data_ref_id_l})
df_title = pd.DataFrame({"title":data_title})
df_url = pd.DataFrame({"url":data_url})

li = pd.concat([df_l_id, df_ref_id_l, df_title, df_url], axis=1)

li.to_csv("link.csv", index=False)

In [14]:
# Reactions
# left/right element 의 drugbank_id는 drug table에 존재하는 것도있고 그렇지 않는것도 있다.

data_r_id = []
data_sequence = []
data_LE_id = []
data_LE_name = []
data_RE_id = []
data_RE_name = []

for child in root.findall("{http://www.drugbank.ca}drug"):
    id = child.find("{http://www.drugbank.ca}drugbank-id[@primary='true']")

    # print("-------------------")
    # print(id.text)

    reactions = child.find("{http://www.drugbank.ca}reactions")
    # print(reactions.tag)

    for reaction in reactions.findall("{http://www.drugbank.ca}reaction"):
        sequence = reaction.find("{http://www.drugbank.ca}sequence")

        left_element = reaction.find("{http://www.drugbank.ca}left-element")
        left_element_drugbank_id = left_element.find("{http://www.drugbank.ca}drugbank-id")
        left_element_name = left_element.find("{http://www.drugbank.ca}name")

        right_element = reaction.find("{http://www.drugbank.ca}right-element")
        right_element_drugbank_id = right_element.find("{http://www.drugbank.ca}drugbank-id")
        right_element_name = right_element.find("{http://www.drugbank.ca}name")

        data_r_id.append(id.text)
        data_sequence.append(sequence.text)
        data_LE_id.append(left_element_drugbank_id.text)
        data_LE_name.append(left_element_name.text)
        data_RE_id.append(right_element_drugbank_id.text)
        data_RE_name.append(right_element_name.text)

        # print("***")
        # print(id.text)
        # print(sequence.text)
        # print(left_element_drugbank_id.text)
        # print(left_element_name.text)
        # print(right_element_drugbank_id.text)
        # print(right_element_name.text)

df_r_id = pd.DataFrame({"drugbank-id":data_r_id})
df_sequence = pd.DataFrame({"sequence":data_sequence})
df_LE_id = pd.DataFrame({"left_element_id":data_LE_id})
df_LE_name = pd.DataFrame({"left_element_name":data_LE_name})
df_RE_id = pd.DataFrame({"right_element_id":data_RE_id})
df_RE_name = pd.DataFrame({"right_element_name":data_RE_name})

ra = pd.concat([df_r_id,df_sequence,df_LE_id,df_LE_name,df_RE_id,df_RE_name], axis=1)
ra.to_csv("Reactions.csv", index=False)
# print(ra.head(5))